In [ ]:
import argparse
import os
import sys
import math

import torch
from torch import nn
from torch import optim

import numpy as np
import pandas as pd
from tqdm import tqdm
import json


## vis
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
def predict(x, a, mu):
    '''UMAP-inspired predict function
    x - torch tensor, shape [n_data_points, n_features]
    a - torch tensor, shape [n_features]
        1/a.abs() is the extent of bounding box at prediction=0.5
    mu - torch tensor, shape [n_features]
    b - scalar. hyper parameter for predict function. Power exponent
    '''

    b = 3
    return 1 / (1 + ((a.abs() * (x - mu).abs()).pow(b)).sum(1))

# test: UMAP-inspired predict function
# n = 100
# x = torch.linspace(-3,3,n).view(n,1)
# a = torch.tensor(0.5)
# plt.plot(x, predict(x, a))


def compute_predicate(x0, selected, n_iter=1000, mu_init=None, a_init=0.4):
    '''
        x0 - numpy array, shape=[n_points, n_feature]. Data points
        selected - boolean array. shape=[n_points] of selection
    '''

    # prepare training data
    # orginal data extent
    n_points, n_features = x0.shape
    vmin = x0.min(0)
    vmax = x0.max(0)
    x = torch.from_numpy(x0.astype(np.float32))
    label = torch.from_numpy(selected).float()
    # normalize
    mean = x.mean(0)
    scale = x.std(0) + 0.1
    x = (x - mean) / scale

    # Trainable parameters
    # since data is normalized,
    # mu can initialized around mean_pos examples
    # a can initialized around a constant across all axes
    center_selected = x[selected].mean(0)
    if mu_init is None:
        mu_init = center_selected
    a = (a_init + 0.1*(2*torch.rand(n_features)-1))
    mu = mu_init + 0.1 * (2*torch.rand(x.shape[1]) - 1)
    a.requires_grad_(True)
    mu.requires_grad_(True)

    # weight-balance selected vs. unselected based on their size
    n_selected = selected.sum()
    n_unselected = n_points - n_selected
    instance_weight = torch.ones(x.shape[0])
    instance_weight[selected] = n_points/n_selected
    instance_weight[~selected] = n_points/n_unselected
    bce = nn.BCELoss(weight=instance_weight)
    optimizer = optim.SGD([
        {'params': mu, 'weight_decay': 0},
        # smaller a encourages larger reach of the bounding box
        {'params': a, 'weight_decay': 0.01}
    ], lr=1e-2, momentum=0.9)

    # training loop
    for e in range(n_iter):
        pred = predict(x, a, mu)
        loss = bce(pred, label)
        loss += (mu - center_selected).pow(2).mean() * 20
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if e % (n_iter//5) == 0:
            # print(pred.min().item(), pred.max().item())
            print(f'[{e:>4}] loss {loss.item()}')
    a.detach_()
    mu.detach_()
    # plt.stem(a.abs().numpy()); plt.show()

    pred = (pred > 0.5).float()
    correct = (pred == label).float().sum().item()
    total = selected.shape[0]
    accuracy = correct/total
    # 1 meaning points are selected
    tp = ((pred == 1).float() * (label == 1).float()).sum().item()
    fp = ((pred == 1).float() * (label == 0).float()).sum().item()
    fn = ((pred == 0).float() * (label == 1).float()).sum().item()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1 = 1/(1/precision + 1/recall)
    print(f'''
accuracy = {correct/total}
precision = {precision}
recall = {recall}
f1 = {f1}
    ''')

    # predicate clause selection
    # r is the range of the bounding box on each dimension
    # bounding box is defined by the level set of prediction=0.5
    r = 1 / a.abs()
    predicates = []
    for k in range(mu.shape[0]):
        # denormalize
        r_k = (r[k] * scale[k]).item()
        mu_k = (mu[k] * scale[k] + mean[k]).item()
        ci = [mu_k - r_k, mu_k + r_k]
        assert ci[0] < ci[1], 'ci[0] is not less than ci[1]'
        if ci[0] < vmin[k]:
            ci[0] = vmin[k]
        if ci[1] > vmax[k]:
            ci[1] = vmax[k]
        # feature selection based on extent range
#         should_include = r[k] < 1.0 * (x[:,k].max()-x[:,k].min())
        should_include = not (ci[0] <= vmin[k] and ci[1] >= vmax[k])
        if should_include:
            predicates.append(dict(
                dim=k, interval=ci
            ))
    for p in predicates:
        print(p)
    return predicates, mu, a, [accuracy, precision, recall, f1]


## predicate test

In [ ]:
df = pd.read_csv(f'./dataset/animals3.csv')
for attr in ['x', 'y', 'image_filename']:
    if attr in df.columns:
        df = df.drop(attr, axis='columns')
x0 = df.to_numpy()

## simulate a sequence of brushes
t = 5
selected = np.zeros([t,x0.shape[0]]).astype(np.bool8)
selected[0, df['big eyes']<0.17] = 1
selected[1, df['big eyes']<0.18] = 1
selected[2, df['big eyes']<0.19] = 1
selected[3, df['big eyes']<0.20] = 1
selected[4, df['big eyes']<0.21] = 1
print('selected', selected.sum(1), 'points in the sequence...')

for i in range(5):
    predicate, mu, a, quality = compute_predicate(x0, selected[i], n_iter=1000, mu_init=None, a_init=0.4)
# predicate, mu, a, quality = compute_predicate_sequence(x0, selected, n_iter=1000, mu_init=None, a_init=0.4)


## Predicate sequence

In [ ]:
from textwrap import dedent

In [ ]:
def predict(x, a, mu):
    '''UMAP-inspired predict function
    x - torch tensor, shape [n_data_points, n_features]
    a - torch tensor, shape [n_features]
        1/a.abs() is the extent of bounding box at prediction=0.5
    mu - torch tensor, shape [n_features]
    b - scalar. hyper parameter for predict function. Power exponent
    '''

    b = 3
    return 1 / (1 + ((a.abs() * (x - mu).abs()).pow(b)).sum(1))

# test: UMAP-inspired predict function
# n = 100
# x = torch.linspace(-3,3,n).view(n,1)
# a = torch.tensor(0.5)
# plt.plot(x, predict(x, a))


def compute_predicate_sequence(x0, selected, n_iter=1000):
    '''
        x0 - numpy array, shape=[n_points, n_feature]. Data points
        selected - boolean array. shape=[brush_index, n_points] of selection
    '''
    mu_init=None
    a_init=0.4
    
    n_points, n_features = x0.shape
    n_brushes = selected.shape[0]
    
    # prepare training data
    # orginal data extent
    vmin = x0.min(0)
    vmax = x0.max(0)
    x = torch.from_numpy(x0.astype(np.float32))
    label = torch.from_numpy(selected).float()
    # normalize
    mean = x.mean(0)
    scale = x.std(0) + 0.1
    x = (x - mean) / scale

    # Trainable parameters
    # since data is normalized,
    # mu can initialized around mean_pos examples
    # a can initialized around a constant across all axes
    selection_centroids = torch.stack([x[sel_t].mean(0) for sel_t in selected], 0)

    if mu_init is None:
        mu_init = selection_centroids
    a = (a_init + 0.1*(2*torch.rand(n_brushes, n_features)-1))
    mu = mu_init + 0.1 * (2*torch.rand(n_brushes, x.shape[1]) - 1)
    a.requires_grad_(True)
    mu.requires_grad_(True)

    # weight-balance selected vs. unselected based on their size
    bce_per_brush = []
    for st in selected: ## for each brush, define their class-balanced loss function
        n_selected = st.sum()
        n_unselected = n_points - n_selected
        instance_weight = torch.ones(x.shape[0])
        instance_weight[st] = n_points/n_selected
        instance_weight[~st] = n_points/n_unselected
        bce = nn.BCELoss(weight=instance_weight)
        bce_per_brush.append(bce)
    
    
    optimizer = optim.SGD([
        {'params': mu, 'weight_decay': 0},
        # smaller a encourages larger reach of the bounding box
        {'params': a, 'weight_decay': 0.01}
    ], lr=1e-2, momentum=0.9)

    # training loop
    for e in range(n_iter):
        loss_per_brush = []
        for t, st in enumerate(selected): ## for each brush, compute loss
            pred = predict(x, a[t], mu[t])
            loss = bce(pred, label[t])
            loss += (mu[t] - selection_centroids[t]).pow(2).mean() * 20
            loss_per_brush.append(loss)
        smoothness_loss = 100 * (a[1:]-a[:-1]).pow(2).mean()
        smoothness_loss += 100 * (mu[1:]-mu[:-1]).pow(2).mean()
        
        total_loss = sum(loss_per_brush) + smoothness_loss
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        if e % (n_iter//5) == 0:
            # print(pred.min().item(), pred.max().item())
            print(f'[{e:>4}] loss {loss.item()}')
    a.detach_()
    mu.detach_()
    # plt.stem(a.abs().numpy()); plt.show()

    qualities = []
    for t, st in enumerate(selected): ## for each brush, compute loss
        pred = predict(x, a[t], mu[t])
        pred = (pred > 0.5).float()
        correct = (pred == label[t]).float().sum().item()
        total = n_points
        accuracy = correct/total
        # 1 meaning points are selected
        tp = ((pred == 1).float() * (label == 1).float()).sum().item()
        fp = ((pred == 1).float() * (label == 0).float()).sum().item()
        fn = ((pred == 0).float() * (label == 1).float()).sum().item()
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1 = 1/(1/precision + 1/recall)
        print(dedent(f'''
            brush = {t}
            accuracy = {accuracy}
            precision = {precision}
            recall = {recall}
            f1 = {f1}
        '''))
        qualities.append(dict(
            brush = t,
            accuracy = accuracy,
            precision = precision,
            recall = recall,
            f1 = f1
        ))

    # predicate clause selection
    # r is the range of the bounding box on each dimension
    # bounding box is defined by the level set of prediction=0.5
    predicates = []
    for t, st in enumerate(selected): ## for each brush, generate a predicate from a[t] and mu[t]
        r = 1 / a[t].abs()
        predicate_clauses = []
        for k in range(n_features): # for each attribute
            # denormalize
            r_k = (r[k] * scale[k]).item()
            mu_k = (mu[t,k] * scale[k] + mean[k]).item()
            ci = [mu_k - r_k, mu_k + r_k]
            assert ci[0] < ci[1], 'ci[0] is not less than ci[1]'
            if ci[0] < vmin[k]:
                ci[0] = vmin[k]
            if ci[1] > vmax[k]:
                ci[1] = vmax[k]
            # feature selection based on extent range
    #         should_include = r[k] < 1.0 * (x[:,k].max()-x[:,k].min())
            should_include = not (ci[0] <= vmin[k] and ci[1] >= vmax[k])
            if should_include:
                predicate_clauses.append(dict(
                    dim=k, interval=ci
                ))
        predicates.append(predicate_clauses)
    parameters = dict(mu=mu, a=a)
    return predicates, qualities, parameters


In [ ]:
df = pd.read_csv(f'./dataset/animals3.csv')
for attr in ['x', 'y', 'image_filename']:
    if attr in df.columns:
        df = df.drop(attr, axis='columns')
x0 = df.to_numpy()

## simulate a sequence of brushes
t = 5
selected = np.zeros([t,x0.shape[0]]).astype(np.bool8)
selected[0, df['big eyes']<0.17] = 1
selected[1, df['big eyes']<0.18] = 1
selected[2, df['big eyes']<0.19] = 1
selected[3, df['big eyes']<0.20] = 1
selected[4, df['big eyes']<0.21] = 1
print('selected', selected.sum(1), 'points in the sequence...')

predicates, qualities, parameters = compute_predicate_sequence(x0, selected, n_iter=100)


In [ ]:
print(predicate)

In [ ]:
# b = 3
# def predict(x, a, mu=0):
#     '''
#     x - torch tensor with shape [n_data_points, n_features]
#     a - torch tensor with shape [n_features]
#     '''
#     return 1 / (1 + ((a.abs() * (x - mu).abs()).pow(b)).sum(1))

# ## test UMAP-inspired predict function
# # n = 100
# # x = torch.linspace(-3,3,n).view(n,1)
# # a = torch.tensor(0.5)
# # plt.plot(x, predict(x, a))


# def predicate(x0, selected, n_iter=3000):
#     '''
#         x0 - numpy array. data points
#         selected - boolean array of selection
#     '''
#     # prepare training data
#     x = torch.from_numpy(x0.astype(np.float32))
#     label = torch.from_numpy(selected).float()
    
#     # normalize
#     mean = x.mean(0)
#     scale = x.std(0) + 0.1
#     x = (x - mean) / scale

#     bce = nn.BCELoss()
    
#     ## since data is normalized, 
#     ## mu can initialized around mean_pos examples
#     ## a can initialized around a constant across all axes
#     mu_init = x[selected].mean(0)
#     a_init = 0.4
    
#     a = (a_init + 0.1*(2*torch.rand(x.shape[1])-1)).requires_grad_(True)
#     mu = mu_init + 0.1 * (2*torch.rand(x.shape[1]) - 1)
#     mu.requires_grad_(True)
#     optimizer = optim.SGD([
#         {'params': mu, 'weight_decay': 0},
#         {'params': a, 'weight_decay': 0.01} ## smaller a encourages larger reach of the bounding box
#     ], lr=1e-2, momentum=0.9)
#     for e in range(n_iter):
#         pred = predict(x, a, mu)
#         l = bce(pred, label)
#         optimizer.zero_grad()
#         l.backward()
#         optimizer.step()
#         if e % (n_iter//5) == 0:
#             print('loss', l.item())
#     a.detach_()
#     mu.detach_()

# #     plt.stem(a.abs().numpy())
# #     plt.show()
    
#     ## range of the bounding box, defined by the level set of prediction=0.5
#     r = 1 / a.abs()
    
#     print(
#         'accuracy',
#         ((pred > 0.5).float() == label).float().sum().item(),
#         '/', selected.shape[0])

    
#     ## orginal data extent
#     vmin = x0.min(0)
#     vmax = x0.max(0)
    
#     ##predicate clause selection
#     predicates = []
#     for k in range(mu.shape[0]):
        
#         ## denormalize
#         r_k = (r[k] * scale[k]).item()
#         mu_k = (mu[k] * scale[k] + mean[k]).item()
#         ci = ((mu_k - r_k), (mu_k + r_k))
        
        
#         ## feature selection based on extent range
# #         should_include = r[k] < 1.0 * (x[:,k].max()-x[:,k].min())
#         should_include = not (ci[0] < vmin[k] and ci[1] > vmax[k])
#         if should_include: 
#             predicates.append(dict(
#                 dim=k, interval=[max(ci[0],vmin[k]), min(ci[1], vmax[k])]
#             ))
#     return predicates




In [ ]:
# ## test, 2D dataset
# data = np.random.rand(150,2)

# target = (
#     (0.3<data[:,0])*(data[:,0]<0.6)
#     *(0<data[:,1])*(data[:,1]<0.5)
# ).astype(np.int64)
# target = ((0.3<data[:,0])*(data[:,0]<0.6)).astype(np.int64)

# for i in range(10):
#     p = predicate(data, target)
#     display(p)

In [ ]:
b = 3
def predict(x, a, mu=0):
    '''
    x - torch tensor with shape [n_data_points, n_features]
    a - torch tensor with shape [n_features]
    '''
    return 1 / (1 + ((a.abs() * (x - mu).abs()).pow(b)).sum(1))

gx, gy = torch.meshgrid(torch.linspace(-3,3,40), torch.linspace(-3,3,40), indexing='xy')
xy = torch.stack([gx.reshape(-1), gy.reshape(-1)], dim=1)
z = predict(xy, a=torch.tensor(1)).reshape([40,40])

gx = gx.numpy()
gy = gy.numpy()
z = z.numpy()

plt.figure(figsize=[4,4])
plt.contour(gx, gy, z)
plt.axis('square');

plt.figure(figsize=[4,2])
plt.plot(gx[0], z[20]);